# 카카오맵 크롤링

search_keyword에 검색할 단어를 입력하고 실행하면 해당 검색 결과에 해당하는 장소들을 크롤링합니다.   
결과는 result 변수.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep


# 네이버 지도는 접속 시 자동으로 아래 url 뒤에 현재 위치 정보가 입력됨
driver_path = 'C:/Users/namke/jupyter workspace/analysis project/Cafe textmining/driver/chromedriver'
URL = 'https://map.kakao.com/'
search_keyword = '포항 이동 카페'

driver = webdriver.Chrome(executable_path=driver_path)
driver.get(URL)

sleep(5)
driver.find_element_by_class_name('DimmedLayer').click() # 배경에 레이어 하나가 떠서 이걸 지우는 코드

# keyword 검색
search = driver.find_element_by_css_selector('div.box_searchbar>input.query')
search.send_keys(search_keyword)
search.send_keys(Keys.ENTER)

sleep(5)
driver.find_element_by_link_text('장소 더보기').send_keys(Keys.ENTER) # div로 인한 click event 오류로 click 대신 send_keys 사용

sleep(5)
# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result = []
i = 1
while True:
    place = soup.select('.PlaceItem')
    for per in place:
        name = per.select_one('.screen_out').text                                          # 장소 이름
        href = per.select_one('.moreview')['href']                                         # 상세보기 주소
        rate = per.select_one('.rating')
        score = rate.select_one('.num').text                                               # 평점
        score_cnt = rate.select_one('.numberofscore').text.replace('건', '')               # 평점 수
        review_cnt = rate.select_one('.review').text.replace('리뷰', '').replace(' ', '')  # 리뷰 수
        addr = per.select_one('.addr').text.replace('\n', '')                              # 장소 주소
        time = per.select_one('.openhour').select_one('a').text                            # 영업 시간
        phone = per.select_one('.phone').text                                              # 전화번호

        # 위 순서대로 저장(column도 동일)
        result.append([name, href, score, score_cnt, review_cnt, addr, time, phone])

    # 페이지 이동
    if i==1 or i%5 != 1:
        try:
            driver.find_element_by_link_text(str(i)).click()
        except:
            break
    else:
        driver.find_element_by_css_selector('#info\.search\.page\.next').click()
    sleep(2)
    i = i + 1

# driver 종료
driver.quit()

C:\Users\namke\AppData\Local\Temp/ipykernel_2460/15438919.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
C:\Users\namke\AppData\Local\Temp/ipykernel_2460/15438919.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('DimmedLayer').click() # 배경에 레이어 하나가 떠서 이걸 지우는 코드
C:\Users\namke\AppData\Local\Temp/ipykernel_2460/15438919.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_css_selector('div.box_searchbar>input.query')
C:\Users\namke\AppData\Local\Temp/ipykernel_2460/15438919.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('장소 더보기').send_keys(Keys.ENTER) # div로 인한 click event 오류로 click 대신 send_keys 사용
C:\Users\namke\AppData

검색결과를 dataframe으로 만들어 확인합니다.

In [4]:
# 크롤링 한 데이터 저장
df = pd.DataFrame(result)
df.columns = ['이름', '링크', '평점', '평점 수 ', '리뷰 수', '주소', '영업시간', '전화번호']
# df.to_csv('result_crawling.csv', index=False, encoding='cp949')
df

,이름,링크,평점,평점 수,리뷰 수,주소,영업시간,전화번호
0,놀숲 포항이동점,https://place.map.kakao.com/404943778,5.0,2,3,경북 포항시 남구 대이로127번길 9 5층(지번) 이동 661,,054-277-2771
1,하삼동커피 포항이동점,https://place.map.kakao.com/38109560,0.0,0,10,경북 포항시 남구 포스코대로 135(지번) 이동 661-4,"토,일 09:00 ~ 22:30",054-612-7177
2,메가커피 포항이동점,https://place.map.kakao.com/621444216,0.0,0,1,경북 포항시 남구 포스코대로 141(지번) 이동 664-17,"토,일 09:30 ~ 22:30",054-284-8838
3,워너디,https://place.map.kakao.com/58974285,0.0,0,6,"경북 포항시 남구 대이로127번길 12-3 1,2층(지번) 이동 658-7",,
4,카페헤츠,https://place.map.kakao.com/24920204,3.0,1,2,경북 포항시 남구 대이로189번길 14(지번) 이동 643-7,,054-278-2681
...,...,...,...,...,...,...,...,...
220,커피테라피,https://place.map.kakao.com/13558218,2.0,1,1,경북 포항시 남구 포스코대로 119(지번) 이동 662-8,,
221,트리플에이커피 포항이동점,https://place.map.kakao.com/1825282492,0.0,0,1,경북 포항시 남구 대이로 149 포스코아 1층 106호(지번) 이동 653-5,,070-7808-0330
222,카페아이앤지,https://place.map.kakao.com/538556338,0.0,0,1,경북 포항시 남구 대이로159번길 6-7(지번) 이동 651-7,,
223,타코N카페,https://place.map.kakao.com/1841308559,5.0,3,1,경북 포항시 남구 대이로127번길 22(지번) 이동 657-7,화~일 14:00 ~ 02:00,


결과를 sqlite3 DB에 저장합니다.

In [5]:
import sqlite3

conn = sqlite3.connect('kakaomap_cafe.db')
df.to_sql('cafe_info_table', conn, chunksize=1000)
conn.commit()
conn.close()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


DB에 저장이 잘 되었는지 확인합니다.

In [10]:
conn = sqlite3.connect('kakaomap_cafe.db')
cur = conn.cursor()
check_table = pd.read_sql('select * from cafe_info_table', conn)
 
conn.commit()
conn.close()

check_table.drop(columns=['index'])

,이름,링크,평점,평점 수,리뷰 수,주소,영업시간,전화번호
0,놀숲 포항이동점,https://place.map.kakao.com/404943778,5.0,2,3,경북 포항시 남구 대이로127번길 9 5층(지번) 이동 661,,054-277-2771
1,하삼동커피 포항이동점,https://place.map.kakao.com/38109560,0.0,0,10,경북 포항시 남구 포스코대로 135(지번) 이동 661-4,"토,일 09:00 ~ 22:30",054-612-7177
2,메가커피 포항이동점,https://place.map.kakao.com/621444216,0.0,0,1,경북 포항시 남구 포스코대로 141(지번) 이동 664-17,"토,일 09:30 ~ 22:30",054-284-8838
3,워너디,https://place.map.kakao.com/58974285,0.0,0,6,"경북 포항시 남구 대이로127번길 12-3 1,2층(지번) 이동 658-7",,
4,카페헤츠,https://place.map.kakao.com/24920204,3.0,1,2,경북 포항시 남구 대이로189번길 14(지번) 이동 643-7,,054-278-2681
...,...,...,...,...,...,...,...,...
220,커피테라피,https://place.map.kakao.com/13558218,2.0,1,1,경북 포항시 남구 포스코대로 119(지번) 이동 662-8,,
221,트리플에이커피 포항이동점,https://place.map.kakao.com/1825282492,0.0,0,1,경북 포항시 남구 대이로 149 포스코아 1층 106호(지번) 이동 653-5,,070-7808-0330
222,카페아이앤지,https://place.map.kakao.com/538556338,0.0,0,1,경북 포항시 남구 대이로159번길 6-7(지번) 이동 651-7,,
223,타코N카페,https://place.map.kakao.com/1841308559,5.0,3,1,경북 포항시 남구 대이로127번길 22(지번) 이동 657-7,화~일 14:00 ~ 02:00,
